In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip

--2021-05-06 15:22:21--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.209.33
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.209.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip’

dataset.zip         100%[===================>]  79.08M  15.6MB/s    in 6.3s    

2021-05-06 15:22:28 (12.6 MB/s) - ‘dataset.zip’ saved [82925180/82925180]



In [2]:
!unzip -q dataset.zip -d dataset

In [2]:
!git clone https://github.com/deayalar/deeplearning_unitn.git
%cd /content/deeplearning_unitn

fatal: destination path 'deeplearning_unitn' already exists and is not an empty directory.
/content/deeplearning_unitn


In [3]:
import torch
import torch.optim as optim

from my_datasets.reid_dataset import ReIdDataset

from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

from utils.split_data import ValidationSplitter, TrainingSplitter
from utils.image_utils import imshow

from models.reid_model import ReIdModel, LeNet
import cost_functions
from torch.utils.tensorboard import SummaryWriter


In [4]:
TRAIN_ROOT = "/content/dataset/train"

# This validation set is used to estimate the performance on the final test set
splitter = ValidationSplitter(train_root=TRAIN_ROOT)
train_set, val_estimation_set = splitter.split(train_size=0.75, random_seed=42)

# Create a validation set for training
train_set, val_set = TrainingSplitter().split(train_set, train_size=0.8, random_seed=42)

#Create pytorch Datasets
composed = transforms.Compose([transforms.ToTensor()])
target_composed = None

train_dataset = ReIdDataset(root_dir=TRAIN_ROOT, 
                            images_list=train_set,
                            transform=composed,
                            target_transform=target_composed)
val_dataset = ReIdDataset(root_dir=TRAIN_ROOT, 
                            images_list=val_set,
                            target_transform=target_composed)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

Identities in train set size: 563
Identities in val set size: 188
Train set size: 9745
Validation set size: 3244
Final Train set size: 7796
Training Validation set size: 1949


In [6]:
def get_optimizer(net, lr, wd, momentum):
  optimizer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd, momentum=momentum)
  return optimizer

def test(net, data_loader, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  # Strictly needed if network contains layers which has different behaviours between train and test
  net.eval()
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      # Load data into GPU
      inputs = inputs.to(device)
      targets = targets.to(device)
      # Forward pass
      outputs = net(inputs)

      # Apply the loss
      loss = cost_function(outputs, targets)

      # Better print something
      samples += inputs.shape[0]
      cumulative_loss += loss.item() # Note: the .item() is needed to extract scalars from tensors
      _, predicted = outputs.max(1)
      cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


# TRAINING
def train(model, data_loader, optimizer, cost_function, device='cuda:0'):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  # Strictly needed if network contains layers which has different behaviours between train and test
  model.train()
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    # Load data into GPU
    inputs = inputs.to(device)
    targets = targets.to(device)
      
    # Forward pass
    outputs = model(inputs)

    # Apply the loss
    loss = cost_function(outputs, targets)
      
    # Backward pass
    loss.backward()
    
    # Update parameters
    optimizer.step()
    
    # Zeros the gradient
    optimizer.zero_grad()

    # Better print something, no?
    samples += inputs.shape[0]
    cumulative_loss += loss.item()
    _, predicted = outputs.max(1)
    cumulative_accuracy += predicted.eq(targets).sum().item()

  return cumulative_loss/samples, cumulative_accuracy/samples*100


In [7]:
def main(batch_size=128, 
         device='cuda:0', 
         learning_rate=0.01, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=1, 
         visualization_name='lenet',
         dataset='mnist', 
         norm=False):
  
  # Creates a logger for the experiment
  writer = SummaryWriter(log_dir=f"runs/{visualization_name}")

  #train_loader, val_loader, test_loader = get_data(batch_size=batch_size, 
  #                                                 test_batch_size=batch_size, 
  #                                                 dataset=dataset)
  
  net = LeNet().to(torch.device(device))
  
  optimizer =  get_optimizer(net, learning_rate, weight_decay, momentum)
  
  cost_function = cost_functions.cross_entropy()

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')
  
  # Add values to plots
  writer.add_scalar('Loss/train_loss', train_loss, 0)
  writer.add_scalar('Loss/val_loss', val_loss, 0)
  writer.add_scalar('Accuracy/train_accuracy', train_accuracy, 0)
  writer.add_scalar('Accuracy/val_accuracy', val_accuracy, 0)

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    val_loss, val_accuracy = test(net, val_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')
    
    # Add values to plots
    writer.add_scalar('Loss/train_loss', train_loss, e + 1)
    writer.add_scalar('Loss/val_loss', val_loss, e + 1)
    writer.add_scalar('Accuracy/train_accuracy', train_accuracy, e + 1)
    writer.add_scalar('Accuracy/val_accuracy', val_accuracy, e + 1)

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
  val_loss, val_accuracy = test(net, val_loader, cost_function)
  #test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Validation loss {:.5f}, Validation accuracy {:.2f}'.format(val_loss, val_accuracy))
  #print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # Closes the logger
  writer.close()

In [8]:
main()

Before training:
tensor([1481, 1135, 1358,   64,  766,  304,  647,  959,  621, 1232,  655, 1206,
        1232,  474,  465,  700,  691,  177, 1487,  752,  463,  931,  148, 1039,
        1126,  600, 1095, 1305, 1017, 1085, 1428,  793,   40, 1305, 1247,  178,
        1437, 1301, 1271,  337, 1478,  556,   40,   64,  353, 1174, 1055,  597,
         794,  164,  609,  775,  858,  995, 1101,  794,  700,  897,  830,   93,
          29,  130,  431, 1020])


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/functional.py:114: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


RuntimeError: ignored